In [1]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import datetime
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import time

In [3]:
from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re

In [4]:
import time

import warnings
warnings.filterwarnings('ignore')

In [5]:
from datetime import datetime

current_time = datetime.now()

print(current_time.strftime("%Y%m%d%H%M%S"))

20231218155535


In [6]:
list_url = 'https://map.naver.com/p/search/충주시%20식당?c=10.00,0,0,0,dh'
starRatingReviewList = []
starRatingList = []
reviewList = []
starRatingResturantList = []
restaurantList = []
reviewDataList_contain_star = []
reviewDataList_without_star = []
restaurantsReviewList_star = []
restaurantsReviewList_nonstar = []
breakPoint = False;
s = Service()
options = webdriver.ChromeOption    s()
wd = webdriver.Chrome(service=s,options=options)
wd.get(list_url)
time.sleep(4)

# 페이지 다운
def page_down(num):
    body = wd.find_element(By.TAG_NAME, 'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)



for i in range(1,4):
    
    #페이지 로딩 대기
    time.sleep(2)
    wd.switch_to.default_content()
    wd.switch_to.frame('searchIframe')
    time.sleep(4)
    page_down(50)
    time.sleep(2)

    isStartPoint = True
    for j in range(1, 50):
        try:
            
            # 리뷰 데이터 리스트 초기화
            reviewDataList_contain_star = []
            reviewDataList_without_star = []
            body = wd.find_element(By.CSS_SELECTOR, '#_pcmap_list_scroll_container > ul > li:nth-child(%d) > div.CHC5F > a > div > div > span.TYaxT' %j)
            body.click()
            time.sleep(2)
            # 클릭하게 되면 div sc-1wsjitl OWZjJ가 개발자 도구화면에 생김.
            wd.switch_to.default_content()
            wd.switch_to.frame('entryIframe')
            
            
            starRating = wd.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]').text

            # 정보 수집 끝이므로, 종료.
            if len(starRatingReviewList) >= 80 and len(reviewList) >= 20:
                breakPoint = True
                break

            if '별점' in starRating:
                #별점이 있는 리뷰
                review = wd.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a').text
                review_num = int(re.sub(r'[^0-9]', '', review))
                #별점
                starRatingInsertComma = re.sub(r'[^0-9]', '', starRating)
                starRating = float(starRatingInsertComma[:1] + "." + starRatingInsertComma[1:])      

                # 방문자 리뷰가 100 이하면 컨티뉴
                if review_num < 100:
                    # 여기서 searchIframe으로 바꾼 이유는 현재 entryIframe인 상태에서 컨티뉴가 되면 다음 for루프에서 wd.switch_to.frame('entryIframe')코드에서 에러가 발생하기 때문
                    wd.switch_to.default_content()
                    wd.switch_to.frame('searchIframe')
                    continue
                # 방문자 리뷰의 리스트가 80이상이면, 더이상 추가 x
                if len(starRatingReviewList) >= 80:
                    # 여기서 searchIframe으로 바꾼 이유는 현재 entryIframe인 상태에서 컨티뉴가 되면 다음 for루프에서 wd.switch_to.frame('entryIframe')코드에서 에러가 발생하기 때문
                    wd.switch_to.default_content()
                    wd.switch_to.frame('searchIframe')
                    continue
                
                # 별점이 존재하는 방문자리뷰 리스트에 더한다.
                starRatingReviewList.append(review_num)
                starRatingRestaurantName = wd.find_element(By.XPATH, '//*[@id="_title"]/span[1]').text
                print(starRatingRestaurantName)
                starRatingResturantList.append(starRatingRestaurantName)
                starRatingList.append(starRating)

                isStartPoint = True # 더보기를 눌렀을 때, 별점이 있는 리뷰 데이터를 분류하기 위해서 flag 설정 => True 별점이 있다. false 별점이 없다.
            else:
                # 별점이 없는 리뷰
                review = wd.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(1) > a').text
                review_num = int(re.sub(r'[^0-9]', '', review))

                # 방문자 리뷰가 100 이하면 컨티뉴
                if review_num < 100:
                    # 여기서 searchIframe으로 바꾼 이유는 현재 entryIframe인 상태에서 컨티뉴가 되면 다음 for루프에서 wd.switch_to.frame('entryIframe')코드에서 에러가 발생하기 때문
                    wd.switch_to.default_content()
                    wd.switch_to.frame('searchIframe')
                    continue
                    # 별점이 존재하지 않는 방문자 리뷰의 리스트가 20이상이면, 더이상 추가 x
                if len(reviewList) >= 20:
                    # 여기서 searchIframe으로 바꾼 이유는 현재 entryIframe인 상태에서 컨티뉴가 되면 다음 for루프에서 wd.switch_to.frame('entryIframe')코드에서 에러가 발생하기 때문
                    wd.switch_to.default_content()
                    wd.switch_to.frame('searchIframe')
                    continue
                # 별점이 존재하지 않는 방문자 리뷰에 더한다.
                reviewList.append(review_num)
                restaurantName = wd.find_element(By.XPATH, '//*[@id="_title"]/span[1]').text
                print(restaurantName)
                restaurantList.append(restaurantName)

                isStartPoint = False # 더보기를 눌렀을 때, 별점이 있는 리뷰 데이터를 분류하기 위해서 flag 설정

            time.sleep(2)
            # 리뷰 버튼 클릭
            wd.find_element(By.CSS_SELECTOR, f'a[href*="review?entry"]').click()

            cnt = 0
            # 리뷰 페이지를 내리면서 더보기로 갱신을 반복
            while True:
                try:
                    cnt += 1
                    time.sleep(1) 
                    wd.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
                    time.sleep(3)
                    wd.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div:nth-child(3) > div.place_section.k5tcc > div.NSTUp > div > a').click() 
                    time.sleep(3) 
                    wd.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
                    time.sleep(1)
                    print(cnt)
                    # 리뷰 데이터가 워낙 많으므로 100개만 수집
                    if cnt == 10:
                        print("리뷰 데이터 100개 수집")
                        break
                except NoSuchElementException: 
                    print('-더보기 버튼 모두 클릭 완료-')
                    break
            
            reviewData = wd.find_elements(By.CLASS_NAME, 'YeINN')
            print(reviewData)

            for review in reviewData:
                try:
                    print(cnt)
                    div_VYGLG = review.find_element(By.CLASS_NAME, 'VYGLG').text
                    
                    # 리뷰 작성자
                    review_writer = div_VYGLG
                    
                    #  리뷰 데이터, 데이터가 없을시 NoSuchElementException 에러
                    span_zPfVt = review.find_element(By.CLASS_NAME, 'zPfVt').text
                    description = span_zPfVt
                        
                    processed_reviewData = (review_writer, description)

                    if isStartPoint == True:
                        reviewDataList_contain_star.append(processed_reviewData)
                    if isStartPoint == False:
                        reviewDataList_without_star.append(processed_reviewData)
                except NoSuchElementException:
                    # 사진만 올라온 경우
                    description = ''
                    processed_reviewData = (review_writer, description)           
                    if isStartPoint == True:
                        reviewDataList_contain_star.append(processed_reviewData)
                    if isStartPoint == False:
                        reviewDataList_without_star.append(processed_reviewData)

            print('a')
            #하나의 모아진 가게 데이터 리뷰를 전체 가게 리뷰 리스트로 넣는다.
            restaurantsReviewList_star.append(reviewDataList_contain_star)
            restaurantsReviewList_nonstar.append(reviewDataList_without_star)
            print('b')
            wd.switch_to.default_content()
            wd.switch_to.frame('searchIframe')
            print('c')
        except:
            wd.switch_to.default_content()
            wd.switch_to.frame('searchIframe')
            print("none")

    # 정보 수집 끝일시, 종료
    if breakPoint == True:
        break
    # 페이지 이동
    
    wd.find_element(By.CSS_SELECTOR, '#app-root > div > div.XUrfU > div.zRM9F > a:nth-child(%d)' %(i+2)).click()
    
print("별점이 없는 방문자 리뷰 리스트")
print(reviewList)
print(restaurantList)
print("별점이 존재하는 방문자 리뷰 리스트")
print(starRatingReviewList)
print(starRatingResturantList)

돈이 조아
1
2
3
4
5
6
7
8
9
10
리뷰 데이터 100개 수집
[<selenium.webdriver.remote.webelement.WebElement (session="0a2620ddb17c180411abf85f29db236d", element="33634241625AA53FDD29001326D1F0F7_element_276")>, <selenium.webdriver.remote.webelement.WebElement (session="0a2620ddb17c180411abf85f29db236d", element="33634241625AA53FDD29001326D1F0F7_element_277")>, <selenium.webdriver.remote.webelement.WebElement (session="0a2620ddb17c180411abf85f29db236d", element="33634241625AA53FDD29001326D1F0F7_element_278")>, <selenium.webdriver.remote.webelement.WebElement (session="0a2620ddb17c180411abf85f29db236d", element="33634241625AA53FDD29001326D1F0F7_element_279")>, <selenium.webdriver.remote.webelement.WebElement (session="0a2620ddb17c180411abf85f29db236d", element="33634241625AA53FDD29001326D1F0F7_element_280")>, <selenium.webdriver.remote.webelement.WebElement (session="0a2620ddb17c180411abf85f29db236d", element="33634241625AA53FDD29001326D1F0F7_element_281")>, <selenium.webdriver.remote.webelement.WebElem

In [7]:
print("별점이 없는 방문자 리뷰 갯수")
print(reviewList)
print("별점이 없는 가게 이름")
print(restaurantList)
print("별점이 없는 가게 갯수")
print(len(reviewList))
print(len(restaurantList))
print("별점이 존재하는 방문자 리뷰 갯수")
print(starRatingReviewList)
print("별점이 존재하는 가게 이름")
print(starRatingResturantList)
print("별점이 존재하는 가게 별점들")
print(starRatingList)
print("별점이 존재하는 가게 갯수")
print(len(starRatingList))
print(len(starRatingReviewList))
print(len(starRatingResturantList))


별점이 없는 방문자 리뷰 갯수
[165, 1735, 211, 1017, 994, 486, 199, 383, 734, 538, 192, 419, 199, 1330, 467, 763, 234, 148, 1020, 409]
별점이 없는 가게 이름
['돈이 조아', '치히로 충주점', '삼산회관 충주호암점', '카쿠레가 충주점', '터줏골명가', '이름난순대의신 본점', '갱고개', '빠리방앗간', '투파인드피터 충주연수점', '준경양카츠', '육삼월', '소인애 낙지냉면', '까사피에스타 충주점', '쇼쿠스시 충주호암점', '권고집만두', '어식당', '재경이네', '아메리칸 버거', '남한강막국수', '비건채식뷔페 빛가람']
별점이 없는 가게 갯수
20
20
별점이 존재하는 방문자 리뷰 갯수
[1321, 687, 2500, 379, 141, 229, 2101, 267, 1507, 367, 1133, 537, 551, 1388, 452, 282, 2135, 996, 1093, 512, 326, 574, 1413, 676, 494, 336, 605, 414, 469, 251, 741, 674, 325, 214, 275, 678, 739, 962, 358, 688, 796, 170, 556, 1084, 360, 351, 484, 684, 207, 1249, 420, 112, 643, 449, 409, 830, 517, 182, 347, 280, 240, 914, 330, 1091, 1409, 587, 301, 185, 324, 1291, 260, 328, 573, 120, 234, 343, 100, 584, 416, 306]
별점이 존재하는 가게 이름
['후라토식당 충주점', '숲속장수촌', '중앙탑메밀마당', '본가할매숨두부 꿩요리', '서로', '수안보 명화식당 꿩요리', '중앙탑막국수', '대장군', '충주홍두깨칼국수보쌈 원조본점', '영화식당', '갈마가든', '카츠집', '상촌식당', '탄금대왕갈비탕', '스마일만두', '플랩잭팬트리 충주점', '원조중앙탑막국

In [23]:
print(restaurantsReviewList_nonstar[1])

[]


In [33]:
print(len(restaurantsReviewList_nonstar))
print(len(restaurantsReviewList_star))

97
97


In [43]:
result_star = []
result_nonstar= []
for i in range(0, 97):
    if len(result_nonstar) == 20:
        break
    if restaurantsReviewList_nonstar[i] == []:
        continue
    result_nonstar.append(restaurantsReviewList_nonstar[i])
for i in range(0, 97):
    if len(result_nonstar) == 80:
        break
    if restaurantsReviewList_star[i] == []:
        continue
    result_star.append(restaurantsReviewList_star[i])

result_nonstar.append(restaurantsReviewList_nonstar[0])
result_star.append(restaurantsReviewList_star[1])
result_star.append(restaurantsReviewList_star[2])
result_star.append(restaurantsReviewList_star[3])

In [44]:
print(len(result_star))
print(len(result_nonstar))

80
20


In [41]:
import csv
csv_file_path = 'output.csv'

with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    for sublist in restaurantsReviewList:
        # Write the header row for each sublist
        csv_writer.writerow(['Author', 'Review'])
        
        # Write data from the current sublist
        csv_writer.writerows(sublist)
        
        # Write an empty row to separate the data between sublists
        csv_writer.writerow([])

NameError: name 'restaurantsReviewList' is not defined

In [45]:
import pandas as pd
# restaurantsReviewList_star, restaurantsReviewList_nonstar에는 별점이 있는 가게의 리뷰데이터, 없는 가게의 리뷰데이터를 분리하여 순서대로 배치했습니다. 

df = pd.DataFrame({'가게 이름': restaurantList, '리뷰 수': reviewList, '리뷰 데이터': result_nonstar})
df

,가게 이름,리뷰 수,리뷰 데이터
0,돈이 조아,165,"[(홍남매맘맘, 고기는 진짜 끝내줘요 두 번째 갔는데 신랑이 너무 너 무 좋아요 집..."
1,치히로 충주점,1735,"[(낑깡9570, 김치나베돈가스는 보글보글 잘 익은 새콤한 김치찌개 에 맛있는 돈가..."
2,삼산회관 충주호암점,211,"[(이성주30, 카이센동이랑 베이컨 크림 우동 먹었는데 둘 다 너무 맛있었어요! ㅠ..."
3,카쿠레가 충주점,1017,"[(시봄, 시청 앞 맛집이라고 해서 가봤는데, 짜글이 맛있게 먹었습니다. 고기도 많..."
4,터줏골명가,994,"[(홍남매맘맘, 맛있어요 자극적이지않고 구수하니 !! 양도 옴청 많아 요~~~ 신랑..."
5,이름난순대의신 본점,486,"[(파란하늘6053, 숙성회가 쫀득하고 맛있어요.), (빈블링, 숙성회 말모말모\n..."
6,갱고개,199,"[(테테림, 트러플 오일 파스타가 젤 마싯었어요 트러플 향이 은 은하게 나서 좋았어..."
7,빠리방앗간,383,"[(봄날568, 스테이크 엄청 부드럽고 육즙도 풍부하고 맛있어요. 파스타도 해물도많..."
8,투파인드피터 충주연수점,734,"[(옌4883, 준경양카츠를 처음 가봤는데, 돈까스가 엄총 바삭바 삭하니 맛있었어요..."
9,준경양카츠,538,"[(야유유, 고기도 직접 구워주시고 반찬도 잘 나오고 맛있고~ 아 이랑 같이 가니 ..."


In [47]:
print(len(starRatingResturantList))
print(len(starRatingReviewList))
print(len(starRatingList))
print(len(result_star))

80
80
80
80


In [51]:
# restaurantsReviewList_star, restaurantsReviewList_nonstar에는 별점이 있는 가게의 리뷰데이터, 없는 가게의 리뷰데이터를 분리하여 순서대로 배치했습니다. 
df2 = pd.DataFrame({'가게 이름': starRatingResturantList, '리뷰 수': starRatingReviewList, '별점': starRatingList, '리뷰 데이터' : result_star})
df2

,가게 이름,리뷰 수,별점,리뷰 데이터
0,후라토식당 충주점,1321,4.68,"[(언젠가한량, 색다른 메뉴에 음식도 맛있어서 또 오고 싶어요. 일본 느낌도 물씬 ..."
1,숲속장수촌,687,4.47,"[(kpearl, 저는 누룽지백숙보다는 닭해물탕이 더 맛있었어요ㅎㅎㅎ), (28**..."
2,중앙탑메밀마당,2500,4.44,"[(sin1190, 오랜만에 지인분과함께 맛있게 먹고 왔네요~ 새로수리해서 깔끔하게..."
3,본가할매숨두부 꿩요리,379,4.42,"[(DesignerH, 처음 먹어보는 꿩 요리! 꿩 특유의 맛이랄 건 크게 없 었지..."
4,서로,141,4.75,"[(킁킁00, 운좋게 예약 자리가 남아서 서로에서 술마실수 있었 어요!! 일단 안주..."
...,...,...,...,...
75,옹기장터,343,4.45,"[(러블리츄츄츄, 충주호암동 호숫가애는 가게앞 전용 주차장이 있어 방문하기 편리했으..."
76,우하하,100,4.55,"[(잎새달열여드레, 막창먹고 싶으면 큰집막창으로 가요. 청국장도 덤으로 맛있고 만족..."
77,메이플라워바베큐,584,4.86,"[(언젠가한량, 색다른 메뉴에 음식도 맛있어서 또 오고 싶어요. 일본 느낌도 물씬 ..."
78,호숫가애,416,4.27,"[(kpearl, 저는 누룽지백숙보다는 닭해물탕이 더 맛있었어요ㅎㅎㅎ), (28**..."


In [50]:
# 저장
df.to_csv('restaurants_nonstar.csv', sep='\t', encoding='utf-8')
df2.to_csv('restaurants_star.csv', sep='\t', encoding='utf-8')


In [185]:
import pandas as pd
df = pd.read_csv('restaurants_nonstar.csv', index_col=0, sep='\t')
print(df)
df.dtypes

           가게 이름  리뷰 수                                             리뷰 데이터
0          돈이 조아   165  [('홍남매맘맘', '고기는 진짜 끝내줘요 두 번째 갔는데 신랑이 너무 너 무 좋아...
1        치히로 충주점  1735  [('낑깡9570', '김치나베돈가스는 보글보글 잘 익은 새콤한 김치찌개 에 맛있는...
2     삼산회관 충주호암점   211  [('이성주30', '카이센동이랑 베이컨 크림 우동 먹었는데 둘 다 너무 맛있었어요...
3       카쿠레가 충주점  1017  [('시봄', '시청 앞 맛집이라고 해서 가봤는데, 짜글이 맛있게 먹었습니다. 고기...
4          터줏골명가   994  [('홍남매맘맘', '맛있어요 자극적이지않고 구수하니 !! 양도 옴청 많아 요~~~...
5     이름난순대의신 본점   486  [('파란하늘6053', '숙성회가 쫀득하고 맛있어요.'), ('빈블링', '숙성회...
6            갱고개   199  [('테테림', '트러플 오일 파스타가 젤 마싯었어요 트러플 향이 은 은하게 나서 ...
7          빠리방앗간   383  [('봄날568', '스테이크 엄청 부드럽고 육즙도 풍부하고 맛있어요. 파스타도 해...
8   투파인드피터 충주연수점   734  [('옌4883', '준경양카츠를 처음 가봤는데, 돈까스가 엄총 바삭바 삭하니 맛있...
9          준경양카츠   538  [('야유유', '고기도 직접 구워주시고 반찬도 잘 나오고 맛있고~ 아 이랑 같이 ...
10           육삼월   192  [('둥실둥실37', '충주 낙지 맛집 소인애 낙지냉면은 저의 최애 낙지 맛 집이랍...
11      소인애 낙지냉면   419  [('북토링', '정말 너무 맛있게 먹고 왔네요. 음식이 맛있으니 먹고 도 기분 좋...
12    까사피에스타 충주점   199  [('ㅎㅇㅎㅇㅎㅇㅎㅇㅎㅇㅇ

가게 이름     object
리뷰 수       int64
리뷰 데이터    object
dtype: object

In [171]:
df2 = pd.read_csv('restaurants_star.csv', index_col=0, sep='\t')
print(df2)
df2.dtypes

          가게 이름  리뷰 수    별점                                             리뷰 데이터
0     후라토식당 충주점  1321  4.68  [('언젠가한량', '색다른 메뉴에 음식도 맛있어서 또 오고 싶어요. 일본 느낌도 ...
1         숲속장수촌   687  4.47  [('kpearl', '저는 누룽지백숙보다는 닭해물탕이 더 맛있었어요ㅎㅎㅎ'), (...
2       중앙탑메밀마당  2500  4.44  [('sin1190', '오랜만에 지인분과함께 맛있게 먹고 왔네요~ 새로수리해서 깔...
3   본가할매숨두부 꿩요리   379  4.42  [('DesignerH', '처음 먹어보는 꿩 요리! 꿩 특유의 맛이랄 건 크게 없...
4            서로   141  4.75  [('킁킁00', '운좋게 예약 자리가 남아서 서로에서 술마실수 있었 어요!! 일단...
..          ...   ...   ...                                                ...
75         옹기장터   343  4.45  [('러블리츄츄츄', '충주호암동 호숫가애는 가게앞 전용 주차장이 있어 방문하기 편...
76          우하하   100  4.55  [('잎새달열여드레', '막창먹고 싶으면 큰집막창으로 가요. 청국장도 덤으로 맛있고...
77     메이플라워바베큐   584  4.86  [('언젠가한량', '색다른 메뉴에 음식도 맛있어서 또 오고 싶어요. 일본 느낌도 ...
78         호숫가애   416  4.27  [('kpearl', '저는 누룽지백숙보다는 닭해물탕이 더 맛있었어요ㅎㅎㅎ'), (...
79     큰집막창 봉방점   306  4.39  [('sin1190', '오랜만에 지인분과함께 맛있게 먹고 왔네요~ 새로수리해서 깔...

[80 rows x 4 columns]


가게 이름      object
리뷰 수        int64
별점        float64
리뷰 데이터     object
dtype: object

In [186]:
temp_list1 = []
temp_list2 = []
temp_list3 = []
for reviews_str in df['가게 이름']:
    temp_list1.append(reviews_str)


for reviews_str in df['리뷰 수']:
    temp_list2.append(reviews_str)

for reviews_str in df['리뷰 데이터']:
    review_list = eval(reviews_str)

    temp_list3.append(review_list)

df = pd.DataFrame({'가게 이름': temp_list1, '리뷰 수': temp_list2, '리뷰 데이터' : temp_list3})


In [187]:
df

,가게 이름,리뷰 수,리뷰 데이터
0,돈이 조아,165,"[(홍남매맘맘, 고기는 진짜 끝내줘요 두 번째 갔는데 신랑이 너무 너 무 좋아요 집..."
1,치히로 충주점,1735,"[(낑깡9570, 김치나베돈가스는 보글보글 잘 익은 새콤한 김치찌개 에 맛있는 돈가..."
2,삼산회관 충주호암점,211,"[(이성주30, 카이센동이랑 베이컨 크림 우동 먹었는데 둘 다 너무 맛있었어요! ㅠ..."
3,카쿠레가 충주점,1017,"[(시봄, 시청 앞 맛집이라고 해서 가봤는데, 짜글이 맛있게 먹었습니다. 고기도 많..."
4,터줏골명가,994,"[(홍남매맘맘, 맛있어요 자극적이지않고 구수하니 !! 양도 옴청 많아 요~~~ 신랑..."
5,이름난순대의신 본점,486,"[(파란하늘6053, 숙성회가 쫀득하고 맛있어요.), (빈블링, 숙성회 말모말모\n..."
6,갱고개,199,"[(테테림, 트러플 오일 파스타가 젤 마싯었어요 트러플 향이 은 은하게 나서 좋았어..."
7,빠리방앗간,383,"[(봄날568, 스테이크 엄청 부드럽고 육즙도 풍부하고 맛있어요. 파스타도 해물도많..."
8,투파인드피터 충주연수점,734,"[(옌4883, 준경양카츠를 처음 가봤는데, 돈까스가 엄총 바삭바 삭하니 맛있었어요..."
9,준경양카츠,538,"[(야유유, 고기도 직접 구워주시고 반찬도 잘 나오고 맛있고~ 아 이랑 같이 가니 ..."


In [174]:
temp_list1 = []
temp_list2 = []
temp_list3 = []
temp_list4 = []

for reviews_str in df2['가게 이름']:
    temp_list1.append(reviews_str)


for reviews_str in df2['리뷰 수']:
    temp_list2.append(reviews_str)

for reviews_str in df2['별점']:
    temp_list3.append(reviews_str)

for reviews_str in df2['리뷰 데이터']:
    review_list = eval(reviews_str)

    temp_list4.append(review_list)

df2 = pd.DataFrame({'가게 이름': temp_list1, '리뷰 수': temp_list2, '별점': temp_list3, '리뷰 데이터' : temp_list4})

In [175]:
df2

,가게 이름,리뷰 수,별점,리뷰 데이터
0,후라토식당 충주점,1321,4.68,"[(언젠가한량, 색다른 메뉴에 음식도 맛있어서 또 오고 싶어요. 일본 느낌도 물씬 ..."
1,숲속장수촌,687,4.47,"[(kpearl, 저는 누룽지백숙보다는 닭해물탕이 더 맛있었어요ㅎㅎㅎ), (28**..."
2,중앙탑메밀마당,2500,4.44,"[(sin1190, 오랜만에 지인분과함께 맛있게 먹고 왔네요~ 새로수리해서 깔끔하게..."
3,본가할매숨두부 꿩요리,379,4.42,"[(DesignerH, 처음 먹어보는 꿩 요리! 꿩 특유의 맛이랄 건 크게 없 었지..."
4,서로,141,4.75,"[(킁킁00, 운좋게 예약 자리가 남아서 서로에서 술마실수 있었 어요!! 일단 안주..."
...,...,...,...,...
75,옹기장터,343,4.45,"[(러블리츄츄츄, 충주호암동 호숫가애는 가게앞 전용 주차장이 있어 방문하기 편리했으..."
76,우하하,100,4.55,"[(잎새달열여드레, 막창먹고 싶으면 큰집막창으로 가요. 청국장도 덤으로 맛있고 만족..."
77,메이플라워바베큐,584,4.86,"[(언젠가한량, 색다른 메뉴에 음식도 맛있어서 또 오고 싶어요. 일본 느낌도 물씬 ..."
78,호숫가애,416,4.27,"[(kpearl, 저는 누룽지백숙보다는 닭해물탕이 더 맛있었어요ㅎㅎㅎ), (28**..."


In [32]:
# 별점없는 리뷰 데이터들 라벨링
for reviews_str in df['리뷰 데이터']:
    reviews_list = eval(reviews_str)  # 문자열을 파이썬 리스트로 변환
    for review_tuple in reviews_list:
        print(review_tuple[1])


고기는 진짜 끝내줘요 두 번째 갔는데 신랑이 너무 너 무 좋아요 집이랑 거리가 있는 데도 가자고 하네요 집 에서 고기 구워 먹는 그런 느낌이에요 넘나 맛...
찐단골이에요 다른고깃집 갈 엄두도 안나요 항상 잘
먹고 후회없이 즐기고와요^^ 직원들 사장님 너무 친
절하시고~~ 다음에가도 전 여기만가요~^^
충주 연수동 돈이조아는 대로변에 위치해 접근성이 좋고 주차장이 있어 방문하기 편리! 양심적인 사장님 이 암돼지 생고기를 저렴한 가격에 주문즉시 썰...
늘 이용하는 고깃집이에요 여긴 의리로갑니다 사장 님의 좋은고기 준비해주시는 의리를 지키시기에 저도 무조건 여기만 이용해요 어제오늘 의리!!!!! ㅎ ...
임지훈에서 간판만바뀌고 사장님은 그대로에용~.~ 전부터 단골이었지만 고기가 점점더좋아져요.. 진짜 신기한게 여기서 먹으면 고기를 오래익혀도 촉...
고기를먹다보면 느끼할수있는부분을 서비스로주는 짬뽕이 딱 잡아주는게 신의한수^^ 고기도 너무맛있고 기본으로주는반찬에셀프바 완전굿 쌈종류도많...
앞뒤로 수십 번 칼집을 낸 목살은 상당히 부드럽고 맛 있어요 보기에도 신선해 보이는 빛깔과 두툼한 사이 즈 그리고 지글지글 구워지는 고기 굽는 소리...
가족모임으로 방문했는데 신선하고 맛있는 고기와 쌈 야채까지 완벽했던 곳이었어요^^ 짬뽕전골도 서비스 로 주셔서 맛있게 먹고 왔답니다:) 특히 사장님...
여태까지 가본 고깃집중에서 진짜 제일 고기맛이 있 는 고깃집입니다👍 짬뽕을 공짜로 주시고 반찬들도 엄청 맛있어서 갈때마다 진짜 배터질듯이 먹고 ...
쫀득하니 너무 맛있게 잘먹었어요 !
다음에도 방문할께요😀
첨 방문인데 매장도 깔꿈하고 고기가 진짜 맛있어요 
쌈채소 코너가 따로있어서 골라 먹기편해요!! 보이는
곳에 숙성고가 있어 더 믿음이가는 곳이에요~~
너무 맛있게 먹고왔어요!!! 잡내없이 부드럽고 맛있는 고기라 만족했답니다. 같이나온 계란찜도 맛있고 특 히 비빔냉면에 고기 싹 싸먹으면 최고! 쌈도 골...
늘 이용하는 사장님이친절한 맛집이죠 삼겹살을 구워 서 잘게 잘라

In [51]:
# 데이터 라벨링하는데 너무 많은 데이터이기에, 이미 학습된 긍정 분석, 부정 분석을 하는 라이브러리를 가져와 예측을 하는 것으로 대신 라벨링
from transformers import pipeline




c:\Users\82105\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [52]:
sentiment_analyzer = pipeline('sentiment-analysis', model='beomi/kcbert-base')



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [69]:


sentiment_analyzer = pipeline('sentiment-analysis', model='beomi/kcbert-base')

column = []
for reivewes in df['리뷰 데이터']:
    temp_list = []
    for data in reivewes:
        sentiment_result = sentiment_analyzer(data[1], max_length=512, truncation=True)

        predicted_sentiment = sentiment_result[0]['label']

        if predicted_sentiment == 'LABEL_0':
            #부정
            temp_list.append(0)
        else:
            #긍정
            temp_list.append(1)
    column.append(temp_list)
df['긍정 부정'] = column
df
        


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,가게 이름,리뷰 수,리뷰 데이터,긍정 부정
0,돈이 조아,165,"[(홍남매맘맘, 고기는 진짜 끝내줘요 두 번째 갔는데 신랑이 너무 너 무 좋아요 집...","[1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, ..."
1,치히로 충주점,1735,"[(낑깡9570, 김치나베돈가스는 보글보글 잘 익은 새콤한 김치찌개 에 맛있는 돈가...","[1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, ..."
2,삼산회관 충주호암점,211,"[(이성주30, 카이센동이랑 베이컨 크림 우동 먹었는데 둘 다 너무 맛있었어요! ㅠ...","[1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, ..."
3,카쿠레가 충주점,1017,"[(시봄, 시청 앞 맛집이라고 해서 가봤는데, 짜글이 맛있게 먹었습니다. 고기도 많...","[0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, ..."
4,터줏골명가,994,"[(홍남매맘맘, 맛있어요 자극적이지않고 구수하니 !! 양도 옴청 많아 요~~~ 신랑...","[1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, ..."
5,이름난순대의신 본점,486,"[(파란하늘6053, 숙성회가 쫀득하고 맛있어요.), (빈블링, 숙성회 말모말모\n...","[1, 0, 0, 1, 1, 1, 0, 1, 1, 1]"
6,갱고개,199,"[(테테림, 트러플 오일 파스타가 젤 마싯었어요 트러플 향이 은 은하게 나서 좋았어...","[1, 1, 0, 0, 1, 0, 1, 1, 0, 0]"
7,빠리방앗간,383,"[(봄날568, 스테이크 엄청 부드럽고 육즙도 풍부하고 맛있어요. 파스타도 해물도많...","[1, 0, 1, 0, 0, 0, 0, 0, 1, 1]"
8,투파인드피터 충주연수점,734,"[(옌4883, 준경양카츠를 처음 가봤는데, 돈까스가 엄총 바삭바 삭하니 맛있었어요...","[1, 0, 0, 1, 0, 1, 0, 0, 0, 1]"
9,준경양카츠,538,"[(야유유, 고기도 직접 구워주시고 반찬도 잘 나오고 맛있고~ 아 이랑 같이 가니 ...","[0, 1, 1, 1, 1, 1, 0, 0, 0, 1]"


In [70]:
print(len(df['긍정 부정'][3]))
print(len(df['리뷰 데이터'][3]))

110
110


In [71]:
column2 = []
for reivewes in df2['리뷰 데이터']:
    temp_list = []
    for data in reivewes:
        sentiment_result = sentiment_analyzer(data[1], max_length=512, truncation=True)

        predicted_sentiment = sentiment_result[0]['label']

        if predicted_sentiment == 'LABEL_0':
            #부정
            temp_list.append(0)
        else:
            #긍정
            temp_list.append(1)
    column2.append(temp_list)
df2['긍정 부정'] = column2
df2

,가게 이름,리뷰 수,별점,리뷰 데이터,긍정 부정
0,후라토식당 충주점,1321,4.68,"[(언젠가한량, 색다른 메뉴에 음식도 맛있어서 또 오고 싶어요. 일본 느낌도 물씬 ...","[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ..."
1,숲속장수촌,687,4.47,"[(kpearl, 저는 누룽지백숙보다는 닭해물탕이 더 맛있었어요ㅎㅎㅎ), (28**...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, ..."
2,중앙탑메밀마당,2500,4.44,"[(sin1190, 오랜만에 지인분과함께 맛있게 먹고 왔네요~ 새로수리해서 깔끔하게...","[0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, ..."
3,본가할매숨두부 꿩요리,379,4.42,"[(DesignerH, 처음 먹어보는 꿩 요리! 꿩 특유의 맛이랄 건 크게 없 었지...","[1, 0, 1, 1, 1, 1, 0, 0, 1, 1]"
4,서로,141,4.75,"[(킁킁00, 운좋게 예약 자리가 남아서 서로에서 술마실수 있었 어요!! 일단 안주...","[1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...
75,옹기장터,343,4.45,"[(러블리츄츄츄, 충주호암동 호숫가애는 가게앞 전용 주차장이 있어 방문하기 편리했으...","[0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, ..."
76,우하하,100,4.55,"[(잎새달열여드레, 막창먹고 싶으면 큰집막창으로 가요. 청국장도 덤으로 맛있고 만족...","[1, 1, 0, 1, 1, 0, 1, 1, 1, 1]"
77,메이플라워바베큐,584,4.86,"[(언젠가한량, 색다른 메뉴에 음식도 맛있어서 또 오고 싶어요. 일본 느낌도 물씬 ...","[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ..."
78,호숫가애,416,4.27,"[(kpearl, 저는 누룽지백숙보다는 닭해물탕이 더 맛있었어요ㅎㅎㅎ), (28**...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, ..."


In [72]:

DataFrame_contain_list = []

for data in df['리뷰 데이터']:
    proccessd_data = []
    for element in data:
        proccessd_data.append(element[1])
    
    temp_df = pd.DataFrame({'리뷰': proccessd_data})

    DataFrame_contain_list.append(temp_df)
   
    

       

In [73]:
DataFrame_contain_list[0]


,리뷰
0,고기는 진짜 끝내줘요 두 번째 갔는데 신랑이 너무 너 무 좋아요 집이랑 거리가 있는...
1,찐단골이에요 다른고깃집 갈 엄두도 안나요 항상 잘\n먹고 후회없이 즐기고와요^^ 직...
2,충주 연수동 돈이조아는 대로변에 위치해 접근성이 좋고 주차장이 있어 방문하기 편리!...
3,늘 이용하는 고깃집이에요 여긴 의리로갑니다 사장 님의 좋은고기 준비해주시는 의리를 ...
4,임지훈에서 간판만바뀌고 사장님은 그대로에용~.~ 전부터 단골이었지만 고기가 점점더좋...
...,...
105,매장 깨끗해서 다음에 또 들르고 싶어요 맛도 맛이지만 친절도 굿
106,맛집 찾았어요!!! 진짜 너무맛있네!! 재방문의사 백만퍼센트요!!!
107,고기품질이 너무 부드럽고 맛있었습니다 잘먹었어요 또 올게요~
108,고기 너무맛있습니다!!! 입에서 살살녹아요 ㅠㅠㅠㅠ다음에 또올게요!!


In [74]:
for index, value in enumerate(df['긍정 부정']):
    temp_df = DataFrame_contain_list[index]
    temp_df['긍정 부정'] = value
    DataFrame_contain_list[index] = temp_df

confirm_df = DataFrame_contain_list[0]


In [75]:
# 1인경우
confirm_df[confirm_df['긍정 부정'] == 1]

,리뷰,긍정 부정
0,고기는 진짜 끝내줘요 두 번째 갔는데 신랑이 너무 너 무 좋아요 집이랑 거리가 있는...,1
5,고기를먹다보면 느끼할수있는부분을 서비스로주는 짬뽕이 딱 잡아주는게 신의한수^^ 고기...,1
9,쫀득하니 너무 맛있게 잘먹었어요 !\n다음에도 방문할께요😀,1
10,첨 방문인데 매장도 깔꿈하고 고기가 진짜 맛있어요 \n쌈채소 코너가 따로있어서 골라...,1
11,너무 맛있게 먹고왔어요!!! 잡내없이 부드럽고 맛있는 고기라 만족했답니다. 같이나온...,1
12,늘 이용하는 사장님이친절한 맛집이죠 삼겹살을 구워 서 잘게 잘라서 김치말이국수에 얹...,1
19,고기가 양이 많고 보이게 진열되 있어서 믿음이 가요~ 근처라 자주 오는데 맛있어요 ...,1
21,주차편하고 고기 신선하고 쌈채소 무한리필 파절이 맛있고 가성비 끝장판 매장 청결 화...,1
24,평이 좋아서 와봤는데 맛있어요,1
28,사장님 오래오래 장사해주세요 ! 고기 너무 맛있어요!!! 짬뽕국물도 최고고 쌈도 많...,1


In [76]:
# 0인경우
confirm_df[confirm_df['긍정 부정'] == 0]

,리뷰,긍정 부정
1,찐단골이에요 다른고깃집 갈 엄두도 안나요 항상 잘\n먹고 후회없이 즐기고와요^^ 직...,0
2,충주 연수동 돈이조아는 대로변에 위치해 접근성이 좋고 주차장이 있어 방문하기 편리!...,0
3,늘 이용하는 고깃집이에요 여긴 의리로갑니다 사장 님의 좋은고기 준비해주시는 의리를 ...,0
4,임지훈에서 간판만바뀌고 사장님은 그대로에용~.~ 전부터 단골이었지만 고기가 점점더좋...,0
6,앞뒤로 수십 번 칼집을 낸 목살은 상당히 부드럽고 맛 있어요 보기에도 신선해 보이는...,0
7,가족모임으로 방문했는데 신선하고 맛있는 고기와 쌈 야채까지 완벽했던 곳이었어요^^ ...,0
8,여태까지 가본 고깃집중에서 진짜 제일 고기맛이 있 는 고깃집입니다👍 짬뽕을 공짜로 ...,0
13,고기 냄새하나도 안나고 반찬들도 맛있네요 쌈들도 \n신선하고 식구들이 맛있게 먹고갑...,0
14,고기가 너무너무 맛있고 직원분들도 다들 너무 친절 하셨어용 고기 정신 없이 먹느라 ...,0
15,쌈채소 종류가 다양해서 좋고 고기가 양도 많아요!!\n매장도 넓고 깨끗해서 자주와서...,0


In [153]:
# df2 전처리

DataFrame_contain_list2 = []

for data in df2['리뷰 데이터']:
    proccessd_data2 = []
    for element in data:
        proccessd_data2.append(element[1])
    
    temp_df2 = pd.DataFrame({'리뷰': proccessd_data2})

    DataFrame_contain_list2.append(temp_df2)

temp_list = []

# 80개 
# for point in df2['별점']:
#     temp_list.append(point)

# for temp_df in DataFrame_contain_list2:
#     temp_df['별점']: 

# for i, temp_df2 in enumerate(DataFrame_contain_list2):
#     temp_df2['별점'] = df2['별점'][i]

DataFrame_contain_list2[0]
    

[                                                    리뷰
 0    색다른 메뉴에 음식도 맛있어서 또 오고 싶어요. 일본 느낌도 물씬 나고 바삭한 식감...
 1    분기마다 한번씩 오는데 항상 만족합니다. 예전에 네이버 예약시 가라아게 서비스 있었...
 2    와...진짜 너무 맛있어요👍🏻👍🏻😭😭😭 올때마다 매번 실망하지 않고 한결같이 맛있네...
 3    주차는 건물 지하 주차장 이용하면 됩니다. 충주에서 맛있는 규카츠 먹고 싶을 때 가...
 4    직원분들이 진짜 친절하시고 음식도 맛있어요. 오므 라니스랑 가츠도 부드럽고 맛있지만...
 ..                                                 ...
 105               재방문했습니다~ 규카츠 너무 맛있어서 우삼겹 덮밥도 먹어봤어요ㅎㅎ
 106  가보고싶었는데 특별한 맛이었어요🥘\n데이트를 할 때 추천합니다❤️\n고기가 부드럽고...
 107                                       맛있어요 ;-)❤️❤️
 108  규카츠를 일본에서 먹어보고 오랜만에 후라토식당에 서 접해봤는데요 !! 진짜 일본에서...
 109                              너무 맛있어요~~~친절하기 까지 합니다
 
 [110 rows x 1 columns],
                                                     리뷰
 0                         저는 누룽지백숙보다는 닭해물탕이 더 맛있었어요ㅎㅎㅎ
 1    급 검색해서 간곳인데 맛있어요~~~ 국물을 조금 졸일 수록 맛있어요~~ 충주는 처음...
 2    오리 백숙 푸짐하고 맛있습니다~! 사장님도 친절하시고, 충주와서 보양 제대로한 느낌...
 3           이것만을 위해 충주에 왔다고 해도 무방해요.. ㅠㅠ 너무 맛있어요!!!!!!
 4    진짜 맛있어요! 여기 수제비

In [78]:
DataFrame_contain_list2[0]

,리뷰
0,색다른 메뉴에 음식도 맛있어서 또 오고 싶어요. 일본 느낌도 물씬 나고 바삭한 식감...
1,분기마다 한번씩 오는데 항상 만족합니다. 예전에 네이버 예약시 가라아게 서비스 있었...
2,와...진짜 너무 맛있어요👍🏻👍🏻😭😭😭 올때마다 매번 실망하지 않고 한결같이 맛있네...
3,주차는 건물 지하 주차장 이용하면 됩니다. 충주에서 맛있는 규카츠 먹고 싶을 때 가...
4,직원분들이 진짜 친절하시고 음식도 맛있어요. 오므 라니스랑 가츠도 부드럽고 맛있지만...
...,...
105,재방문했습니다~ 규카츠 너무 맛있어서 우삼겹 덮밥도 먹어봤어요ㅎㅎ
106,가보고싶었는데 특별한 맛이었어요🥘\n데이트를 할 때 추천합니다❤️\n고기가 부드럽고...
107,맛있어요 ;-)❤️❤️
108,규카츠를 일본에서 먹어보고 오랜만에 후라토식당에 서 접해봤는데요 !! 진짜 일본에서...


In [79]:
for index, value in enumerate(df2['긍정 부정']):
    temp_df2 = DataFrame_contain_list2[index]
    temp_df2['긍정 부정'] = value
    DataFrame_contain_list2[index] = temp_df2

confirm_df2 = DataFrame_contain_list2[0]

In [80]:
# 1인경우
confirm_df2[confirm_df2['긍정 부정'] == 1]

,리뷰,긍정 부정
2,와...진짜 너무 맛있어요👍🏻👍🏻😭😭😭 올때마다 매번 실망하지 않고 한결같이 맛있네...,1
4,직원분들이 진짜 친절하시고 음식도 맛있어요. 오므 라니스랑 가츠도 부드럽고 맛있지만...,1
8,충주 오는 길에 맛집 검색해서 왔어요. 각자 하나씩 구워가며 재미나게 먹었답니다. ...,1
10,"규카츠 우동 너무 맛있어요!! 규카츠는 취향에 맞게 구\n워먹으니 맛있고, 우동은 ...",1
15,오므라이스 넘 부드럽고 맛나요~^^\n규카츠도 말해 뭐해요~ㅎㅎ \n눈길을 뚫고 충...,1
...,...,...
105,재방문했습니다~ 규카츠 너무 맛있어서 우삼겹 덮밥도 먹어봤어요ㅎㅎ,1
106,가보고싶었는데 특별한 맛이었어요🥘\n데이트를 할 때 추천합니다❤️\n고기가 부드럽고...,1
107,맛있어요 ;-)❤️❤️,1
108,규카츠를 일본에서 먹어보고 오랜만에 후라토식당에 서 접해봤는데요 !! 진짜 일본에서...,1


In [81]:
# 0인경우
confirm_df2[confirm_df2['긍정 부정'] == 0]

,리뷰,긍정 부정
0,색다른 메뉴에 음식도 맛있어서 또 오고 싶어요. 일본 느낌도 물씬 나고 바삭한 식감...,0
1,분기마다 한번씩 오는데 항상 만족합니다. 예전에 네이버 예약시 가라아게 서비스 있었...,0
3,주차는 건물 지하 주차장 이용하면 됩니다. 충주에서 맛있는 규카츠 먹고 싶을 때 가...,0
5,"돈까스를 개인화로에서 직접 구워먹는 독특한 컨셉의 일식 식당입니다. 주차 편리하고,...",0
6,"규카츠 먹고싶어서 방문했어요~ 음식들이 맛있는건 물론이고, 엄청 친절하세요!! 부모...",0
7,일단 직원분들이 너무 친절하세요! 그리고 충주에 이\n런 맛집이 있다니 고기도 너무...,0
9,토요일 오픈시간에 딱 맞춰왔는데 다들 벌써 입장대 기하고 계시고있고..!! 오픈 십...,0
11,지인추천으로 왔어요~ 역시 충주 맛집이네요! 아기의자 2개있어요~ 분위가 조용하고 ...,0
12,"우리 5가족 방문해서 스테끼정식 더블, 규카츠 더블, \n오므라이스 시켰는데 깔끔하...",0
13,일본에서 맛보았던 규카츠를 여기서도 먹을 수 있다 는 기쁨에 진짜 잔뜩 기대하고 감...,0


In [213]:
3. 각 식당의 리뷰 데이터에 대해 감성분석을 수행하여 전체 리뷰수에 대한 긍정 리뷰수의 비율을 구하시오. 긍정 리뷰수 비율을 포함한 데이터프레임을 만드시오. 
   - 별점이 있는 데이터 : 식당명, 위치(시), 별점, 리뷰수, 긍정리뷰비율
   - 별점이 없는 데이터 : 식당명, 위치(시), 리뷰수, 긍정리뷰비율


SyntaxError: invalid syntax (1047931970.py, line 1)

In [82]:
# 
DataFrame_contain_list[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   리뷰      110 non-null    object
 1   긍정 부정   110 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.8+ KB


In [83]:
print(DataFrame_contain_list[0]['긍정 부정'].value_counts())
# 긍정이 35 부정이 75개다.

긍정 부정
1    58
0    52
Name: count, dtype: int64


In [85]:
import re
for index, value in enumerate(DataFrame_contain_list):
    DataFrame_contain_list[index]['리뷰'] = DataFrame_contain_list[index]['리뷰'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))


for index, value in enumerate(DataFrame_contain_list2):
    DataFrame_contain_list2[index]['리뷰'] = DataFrame_contain_list2[index]['리뷰'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))


In [86]:
print(len(DataFrame_contain_list))
print(len(DataFrame_contain_list2))

20
80


In [87]:
from konlpy.tag import Okt

okt = Okt()

In [88]:
def okt_tokenizer(text):
    tokens = okt.morphs(text)
    return tokens

In [89]:
from sklearn.model_selection import train_test_split

In [90]:
# 별점없는 데이터 훈련세트와 테스트 세트로 분리, 별점있는 데이터 훈련세트와 테스트 세트로 분리 
df_contain_train_list = []
df_contain_test_list = [] 
df2_contain_train_list = []
df2_contain_test_list = []

for index, value in enumerate(DataFrame_contain_list):
    DataFrame_contain_list_train, DataFrame_contain_list_test = train_test_split(DataFrame_contain_list[index], test_size=0.2, random_state=42)

    df_contain_train_list.append(DataFrame_contain_list_train)
    df_contain_test_list.append(DataFrame_contain_list_test) 

for index, value in enumerate(DataFrame_contain_list2):
    DataFrame_contain_list2_train, DataFrame_contain_list2_test = train_test_split(DataFrame_contain_list2[index], test_size=0.2, random_state=42)

    df2_contain_train_list.append(DataFrame_contain_list2_train)
    df2_contain_test_list.append(DataFrame_contain_list2_test)



In [91]:
print(len(df_contain_train_list))
print(len(df2_contain_train_list))

20
80


In [92]:
collected_train_List = []
collected_train_List2 = []
collected_train_emotion_List = []
collected_train_emotion_List2 = []
for df_train in df_contain_train_list:
    for element in df_train['리뷰']:
    
        collected_train_List.append(element)
    for element in df_train['긍정 부정']:
        collected_train_emotion_List.append(element)
        
for df2_train in df2_contain_train_list:
    for element in df2_train['리뷰']:
        
        collected_train_List2.append(element)
    for element in df2_train['긍정 부정']:
        collected_train_emotion_List2.append(element)

print(len(collected_train_List))
print(len(collected_train_List2))
print(len(collected_train_emotion_List))
print(len(collected_train_emotion_List2))

fin_train_df = pd.DataFrame({'리뷰': collected_train_List, '긍정 부정': collected_train_emotion_List})
fin_train_df2 = pd.DataFrame({'리뷰': collected_train_List2, '긍정 부정': collected_train_emotion_List2})


880
2114
880
2114


In [93]:
collected_test_emotion_List = []
collected_test_emotion_List2 = []
collected_test_List = []
collected_test_List2 = []

for df_test in df_contain_test_list:
    for element in df_test['긍정 부정']:        
        collected_test_emotion_List.append(element)
    for element in df_test['리뷰']:
        collected_test_List.append(element)
        
for df2_test in df2_contain_test_list:
    for element in df2_test['긍정 부정']:     
        collected_test_emotion_List2.append(element)
    for element in df2_test['리뷰']:
        collected_test_List2.append(element)

print(len(collected_test_List))
print(len(collected_test_List2))
print(len(collected_test_emotion_List))
print(len(collected_test_emotion_List2))

fin_test_df = pd.DataFrame({'리뷰' : collected_test_List, '긍정 부정': collected_test_emotion_List})
fin_test_df2 = pd.DataFrame({'리뷰' : collected_test_List2, '긍정 부정': collected_test_emotion_List2})

220
531
220
531


In [94]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer = okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf.fit(fin_train_df['리뷰'])
non_star_train_tfidf = tfidf.transform(fin_train_df['리뷰'])


c:\Users\82105\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [95]:
print(non_star_train_tfidf)


  (0, 1195)	0.476363391868921
  (0, 1041)	0.46546230198839583
  (0, 848)	0.21219669596639656
  (0, 842)	0.48894777658110744
  (0, 426)	0.31750530601097005
  (0, 2)	0.41414463532323653
  (1, 1310)	0.2718848467790829
  (1, 1161)	0.5437696935581658
  (1, 1061)	0.25706188186129436
  (1, 1036)	0.2511792831417899
  (1, 902)	0.28171522406660016
  (1, 867)	0.19925475636243242
  (1, 707)	0.14865327347679372
  (1, 706)	0.14811927844727488
  (1, 705)	0.28171522406660016
  (1, 704)	0.25706188186129436
  (1, 586)	0.17709100592425728
  (1, 275)	0.14865327347679372
  (1, 234)	0.11377225496306657
  (1, 160)	0.2066141654512811
  (1, 102)	0.2891045265483021
  (2, 1383)	0.13624448197785954
  (2, 1336)	0.16675135407959824
  (2, 1335)	0.1576976260543873
  (2, 1291)	0.13087523990036254
  :	:
  (877, 321)	0.37284315043962984
  (877, 286)	0.32521044786361264
  (877, 284)	0.1924705559604943
  (877, 267)	0.21255409523447008
  (878, 1372)	0.3221540027636421
  (878, 1032)	0.36131836167905723
  (878, 1001)	0.31549

In [96]:
from sklearn.linear_model import LogisticRegression

SA_lr = LogisticRegression(random_state = 0)

In [97]:
SA_lr.fit(non_star_train_tfidf, fin_train_df['긍정 부정'])

LogisticRegression(random_state=0)

In [98]:
from sklearn.model_selection import GridSearchCV

params = {'C': [1, 3, 3.5, 4, 4.5, 5]}
SA_lr_grid_cv = GridSearchCV(SA_lr, param_grid=params, cv=3, scoring='accuracy', verbose=1)

In [99]:
SA_lr_grid_cv.fit(non_star_train_tfidf, fin_train_df['긍정 부정'])

Fitting 3 folds for each of 6 candidates, totalling 18 fits


GridSearchCV(cv=3, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1, 3, 3.5, 4, 4.5, 5]}, scoring='accuracy',
             verbose=1)

In [100]:
print(SA_lr_grid_cv.best_params_, round(SA_lr_grid_cv.best_score_, 4))

{'C': 5} 0.7363


In [101]:
SA_lr_best = SA_lr_grid_cv.best_estimator_

In [102]:
non_star_test_tfidf = tfidf.transform(fin_test_df['리뷰'])

In [103]:
test_predict = SA_lr_best.predict(non_star_test_tfidf)

In [104]:
from sklearn.metrics import accuracy_score

print('감성 분석 정확도 : ', round(accuracy_score(fin_test_df['긍정 부정'], test_predict), 3))

감성 분석 정확도 :  0.655


In [105]:
# DataFrame_contain_list = []

for restarunt_df in DataFrame_contain_list:
    print(restarunt_df)

                                                    리뷰  긍정 부정
0    고기는 진짜 끝내줘요 두 번째 갔는데 신랑이 너무 너 무 좋아요 집이랑 거리가 있는...      1
1    찐단골이에요 다른고깃집 갈 엄두도 안나요 항상 잘먹고 후회없이 즐기고와요 직원들 사...      0
2    충주 연수동 돈이조아는 대로변에 위치해 접근성이 좋고 주차장이 있어 방문하기 편리 ...      0
3    늘 이용하는 고깃집이에요 여긴 의리로갑니다 사장 님의 좋은고기 준비해주시는 의리를 ...      0
4    임지훈에서 간판만바뀌고 사장님은 그대로에용 전부터 단골이었지만 고기가 점점더좋아져요...      0
..                                                 ...    ...
105                매장 깨끗해서 다음에 또 들르고 싶어요 맛도 맛이지만 친절도 굿      1
106                      맛집 찾았어요 진짜 너무맛있네 재방문의사 백만퍼센트요      0
107                   고기품질이 너무 부드럽고 맛있었습니다 잘먹었어요 또 올게요      1
108                  고기 너무맛있습니다 입에서 살살녹아요 ㅠㅠㅠㅠ다음에 또올게요      0
109              이쪽으로 간거 신의한수 ㅠㅠ 맛도 서비스도 양도 폼 미쳤습니다 ㅎㅎ      0

[110 rows x 2 columns]
                                                    리뷰  긍정 부정
0    김치나베돈가스는 보글보글 잘 익은 새콤한 김치찌개 에 맛있는 돈가스가 들어 가서 추...      1
1    텐동 연어장 얼큰김치우동 주문했어요 텐동 얇은 튀 김옷을 입은 파삭한 튀김을 와사비...      0
2    비가 와서 또 치히로 왔어요 김치나베가 급 생각나서ㅋㅋ 연어초밥 신선하고 

In [191]:
fin_train_df2['리뷰']

0       에비카츠동과 스테키 정식을 먹었는데 정말 맛있고 양도 푸짐했어요 직원분들도 친절하시...
1                          다 맛있어요 리뷰이벤트 새우튀김이 진짜 크고 맛있어요짱
2       너무 맛있어요 부드럽고 육즙이 가득해요 이런 메뉴는 처음인데 홀딱 반했어요 아이데리...
3       부드러움이 기본으로 갖추어진 집 진짜 맛있어요 오픈시간부터 자리가 나 채워지니 오픈...
4              고기도부드럽고 진짜 존맛탱이에요 충주 처음 왔는데 정말 맛있게 먹고 갑니당 
                              ...                        
2109                                           치킨이 더 맛있는곳
2110    진짜 치킨이 너무 부드럽고 맛있어서 손이 계속 갔어요 치킨 한마리 반반으로 나눠주셔...
2111                                                     
2112                                            리모델링후첨가봐요
2113                 진짜 바삭하고 맛있는 치킨과 양념장이 신의 한수인 메밀비빔국수에요
Name: 리뷰, Length: 2114, dtype: object

In [106]:
tfidf.fit(fin_train_df2['리뷰'])
star_train_tfidf = tfidf.transform(fin_train_df2['리뷰'])

c:\Users\82105\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [107]:
print(star_train_tfidf)

  (0, 2938)	0.13838304762404097
  (0, 2913)	0.11342227404864956
  (0, 2887)	0.19122783640638852
  (0, 2886)	0.14374161857137446
  (0, 2805)	0.14131789320064903
  (0, 2804)	0.12572964451405194
  (0, 2765)	0.19630645253821152
  (0, 2695)	0.2025221763360143
  (0, 2693)	0.13054422767685328
  (0, 2617)	0.2025221763360143
  (0, 2531)	0.2025221763360143
  (0, 2528)	0.14055468738585655
  (0, 2502)	0.13510216940284628
  (0, 2499)	0.12360225459422443
  (0, 2349)	0.19122783640638852
  (0, 2347)	0.14055468738585655
  (0, 2336)	0.18321437469795743
  (0, 2332)	0.10642184287599507
  (0, 2034)	0.18693392764941721
  (0, 2032)	0.10429445295616759
  (0, 1767)	0.07321845576297434
  (0, 1722)	0.16570431097052893
  (0, 1718)	0.11673870462907217
  (0, 1562)	0.18693392764941721
  (0, 1450)	0.15503595850100027
  :	:
  (2110, 521)	0.09812768918849375
  (2110, 463)	0.1723723054985677
  (2110, 449)	0.07912282668789736
  (2110, 283)	0.16038352122179017
  (2110, 203)	0.16038352122179017
  (2110, 144)	0.189065459013

In [116]:
SA_lr.fit(star_train_tfidf, fin_train_df2['긍정 부정'])


LogisticRegression(random_state=0)

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'C': [1, 3, 3.5, 4, 4.5, 5]}
SA_lr_grid_cv = GridSearchCV(SA_lr, param_grid=params, cv=3, scoring='accuracy', verbose=1)

In [117]:
SA_lr_grid_cv.fit(star_train_tfidf, fin_train_df2['긍정 부정'])

Fitting 3 folds for each of 6 candidates, totalling 18 fits


GridSearchCV(cv=3, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1, 3, 3.5, 4, 4.5, 5]}, scoring='accuracy',
             verbose=1)

In [118]:
print(SA_lr_grid_cv.best_params_, round(SA_lr_grid_cv.best_score_, 4))

{'C': 5} 0.8165


In [119]:
SA_lr_best = SA_lr_grid_cv.best_estimator_

In [120]:
star_test_tfidf = tfidf.transform(fin_test_df2['리뷰'])

In [115]:
print(len(fin_test_df2['리뷰']))
print(len(fin_train_df2['리뷰']))

531
2114


In [121]:
test_predict2 = SA_lr_best.predict(star_test_tfidf)

In [123]:
from sklearn.metrics import accuracy_score

print('감성 분석 정확도 : ', round(accuracy_score(fin_test_df2['긍정 부정'], test_predict2), 3))

감성 분석 정확도 :  0.766


In [126]:
# 별점 없는 리뷰 데이터
for data_frame in DataFrame_contain_list:
    review_tfidf= tfidf.transform(data_frame['리뷰'])
    review_predict = SA_lr_best.predict(review_tfidf)
    data_frame['예측 긍정부정'] = review_predict

DataFrame_contain_list[0]

,리뷰,긍정 부정,예측 긍정부정
0,김치나베돈가스는 보글보글 잘 익은 새콤한 김치찌개 에 맛있는 돈가스가 들어 가서 추...,1,0
1,텐동 연어장 얼큰김치우동 주문했어요 텐동 얇은 튀 김옷을 입은 파삭한 튀김을 와사비...,0,1
2,비가 와서 또 치히로 왔어요 김치나베가 급 생각나서ㅋㅋ 연어초밥 신선하고 좋아요 밥...,1,1
3,연어장 처음 먹어봤는데 맛있었어요호르몬 동이라 는 대창덮밥도 대창이 큼직큼직하게 올...,1,0
4,연어장덮밥 먹고싶어서 방문했는데 맛집이라 그런지 사람이 엄청 많았는데 운좋게 자리 ...,0,0
...,...,...,...
105,호암동 찐맛집너무맛있네요,0,1
106,직원분들이 넘 친절하고 음식도 깔끔하니 맛있어요,1,0
107,음 메뉴 선택을 잘못한것 같아요 사케동 텐동 이런메뉴가 맛있나봐요 사이드로 시킨 연...,1,1
108,음식이 맛있어요 직원들도 엄청 친절하세요,1,1


In [129]:
#별점이 존재하는 리뷰 데이터
for data_frame in DataFrame_contain_list2:
    review_tfidf= tfidf.transform(data_frame['리뷰'])
    review_predict = SA_lr_best.predict(review_tfidf)
    data_frame['예측 긍정부정'] = review_predict

DataFrame_contain_list2

[                                                    리뷰  긍정 부정  예측 긍정부정
 0    색다른 메뉴에 음식도 맛있어서 또 오고 싶어요 일본 느낌도 물씬 나고 바삭한 식감도...      0        0
 1    분기마다 한번씩 오는데 항상 만족합니다 예전에 네이버 예약시 가라아게 서비스 있었는...      0        0
 2    와진짜 너무 맛있어요 올때마다 매번 실망하지 않고 한결같이 맛있네요 규카츠는 겉바속...      1        1
 3    주차는 건물 지하 주차장 이용하면 됩니다 충주에서 맛있는 규카츠 먹고 싶을 때 가는...      0        0
 4    직원분들이 진짜 친절하시고 음식도 맛있어요 오므 라니스랑 가츠도 부드럽고 맛있지만 ...      1        1
 ..                                                 ...    ...      ...
 105                재방문했습니다 규카츠 너무 맛있어서 우삼겹 덮밥도 먹어봤어요ㅎㅎ      1        1
 106  가보고싶었는데 특별한 맛이었어요데이트를 할 때 추천합니다고기가 부드럽고 맛있고 가게...      1        1
 107                                              맛있어요       1        1
 108  규카츠를 일본에서 먹어보고 오랜만에 후라토식당에 서 접해봤는데요  진짜 일본에서 먹...      1        0
 109                                 너무 맛있어요친절하기 까지 합니다      1        1
 
 [110 rows x 3 columns],
                                                     리뷰  긍정 부정  예측 긍정부정
 0                         저는 누룽지백숙보다

In [199]:
temp_list = []
for reviews in df['리뷰 데이터']:
    temp_df = pd.DataFrame({'리뷰': reviews})
    temp_df['리뷰'] = temp_df['리뷰'].apply(lambda x: ' '.join(x))
    temp_df_tfidf = tfidf.transform(temp_df['리뷰'])
    review_predict = SA_lr_best.predict(temp_df_tfidf)
    temp_list.append(review_predict)

df['긍정 부정'] = temp_list





In [201]:
df['긍정 부정의 비율'] = df['긍정 부정'].apply(lambda x: sum(x) / len(x))

In [202]:
df

,가게 이름,리뷰 수,리뷰 데이터,긍정 부정,긍정 부정의 비율
0,돈이 조아,165,"[(홍남매맘맘, 고기는 진짜 끝내줘요 두 번째 갔는데 신랑이 너무 너 무 좋아요 집...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ...",0.490909
1,치히로 충주점,1735,"[(낑깡9570, 김치나베돈가스는 보글보글 잘 익은 새콤한 김치찌개 에 맛있는 돈가...","[0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...",0.627273
2,삼산회관 충주호암점,211,"[(이성주30, 카이센동이랑 베이컨 크림 우동 먹었는데 둘 다 너무 맛있었어요! ㅠ...","[0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...",0.427273
3,카쿠레가 충주점,1017,"[(시봄, 시청 앞 맛집이라고 해서 가봤는데, 짜글이 맛있게 먹었습니다. 고기도 많...","[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, ...",0.781818
4,터줏골명가,994,"[(홍남매맘맘, 맛있어요 자극적이지않고 구수하니 !! 양도 옴청 많아 요~~~ 신랑...","[1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, ...",0.881818
5,이름난순대의신 본점,486,"[(파란하늘6053, 숙성회가 쫀득하고 맛있어요.), (빈블링, 숙성회 말모말모\n...","[1, 1, 1, 1, 0, 1, 1, 1, 1, 1]",0.900000
6,갱고개,199,"[(테테림, 트러플 오일 파스타가 젤 마싯었어요 트러플 향이 은 은하게 나서 좋았어...","[0, 0, 1, 0, 1, 0, 0, 0, 1, 0]",0.300000
7,빠리방앗간,383,"[(봄날568, 스테이크 엄청 부드럽고 육즙도 풍부하고 맛있어요. 파스타도 해물도많...","[1, 0, 0, 0, 0, 1, 0, 1, 0, 0]",0.300000
8,투파인드피터 충주연수점,734,"[(옌4883, 준경양카츠를 처음 가봤는데, 돈까스가 엄총 바삭바 삭하니 맛있었어요...","[1, 1, 0, 1, 1, 1, 1, 1, 0, 1]",0.800000
9,준경양카츠,538,"[(야유유, 고기도 직접 구워주시고 반찬도 잘 나오고 맛있고~ 아 이랑 같이 가니 ...","[0, 1, 0, 1, 1, 1, 0, 0, 0, 0]",0.400000


In [155]:
df2

,가게 이름,리뷰 수,별점,리뷰 데이터,긍정 부정
0,후라토식당 충주점,1321,4.68,"[(언젠가한량, 색다른 메뉴에 음식도 맛있어서 또 오고 싶어요. 일본 느낌도 물씬 ...","[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ..."
1,숲속장수촌,687,4.47,"[(kpearl, 저는 누룽지백숙보다는 닭해물탕이 더 맛있었어요ㅎㅎㅎ), (28**...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, ..."
2,중앙탑메밀마당,2500,4.44,"[(sin1190, 오랜만에 지인분과함께 맛있게 먹고 왔네요~ 새로수리해서 깔끔하게...","[0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, ..."
3,본가할매숨두부 꿩요리,379,4.42,"[(DesignerH, 처음 먹어보는 꿩 요리! 꿩 특유의 맛이랄 건 크게 없 었지...","[1, 0, 1, 1, 1, 1, 0, 0, 1, 1]"
4,서로,141,4.75,"[(킁킁00, 운좋게 예약 자리가 남아서 서로에서 술마실수 있었 어요!! 일단 안주...","[1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...
75,옹기장터,343,4.45,"[(러블리츄츄츄, 충주호암동 호숫가애는 가게앞 전용 주차장이 있어 방문하기 편리했으...","[0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, ..."
76,우하하,100,4.55,"[(잎새달열여드레, 막창먹고 싶으면 큰집막창으로 가요. 청국장도 덤으로 맛있고 만족...","[1, 1, 0, 1, 1, 0, 1, 1, 1, 1]"
77,메이플라워바베큐,584,4.86,"[(언젠가한량, 색다른 메뉴에 음식도 맛있어서 또 오고 싶어요. 일본 느낌도 물씬 ...","[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ..."
78,호숫가애,416,4.27,"[(kpearl, 저는 누룽지백숙보다는 닭해물탕이 더 맛있었어요ㅎㅎㅎ), (28**...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, ..."


In [204]:
temp_list2 = []
for reviews in df2['리뷰 데이터']:
    temp_df2 = pd.DataFrame({'리뷰': reviews})
    temp_df2['리뷰'] = temp_df2['리뷰'].apply(lambda x: ' '.join(x))
    temp_df2_tfidf = tfidf.transform(temp_df2['리뷰'])
    review_predict = SA_lr_best.predict(temp_df2_tfidf)
    temp_list2.append(review_predict)

df2['긍정 부정'] = temp_list2

In [205]:
df2['긍정 부정의 비율'] = df['긍정 부정'].apply(lambda x: sum(x) / len(x))

In [206]:
df2

,가게 이름,리뷰 수,별점,리뷰 데이터,긍정 부정,긍정 부정의 비율
0,후라토식당 충주점,1321,4.68,"[(언젠가한량, 색다른 메뉴에 음식도 맛있어서 또 오고 싶어요. 일본 느낌도 물씬 ...","[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ...",0.490909
1,숲속장수촌,687,4.47,"[(kpearl, 저는 누룽지백숙보다는 닭해물탕이 더 맛있었어요ㅎㅎㅎ), (28**...","[1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, ...",0.627273
2,중앙탑메밀마당,2500,4.44,"[(sin1190, 오랜만에 지인분과함께 맛있게 먹고 왔네요~ 새로수리해서 깔끔하게...","[0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, ...",0.427273
3,본가할매숨두부 꿩요리,379,4.42,"[(DesignerH, 처음 먹어보는 꿩 요리! 꿩 특유의 맛이랄 건 크게 없 었지...","[1, 0, 1, 1, 1, 1, 0, 0, 1, 1]",0.781818
4,서로,141,4.75,"[(킁킁00, 운좋게 예약 자리가 남아서 서로에서 술마실수 있었 어요!! 일단 안주...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...",0.881818
...,...,...,...,...,...,...
75,옹기장터,343,4.45,"[(러블리츄츄츄, 충주호암동 호숫가애는 가게앞 전용 주차장이 있어 방문하기 편리했으...","[0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, ...",NaN
76,우하하,100,4.55,"[(잎새달열여드레, 막창먹고 싶으면 큰집막창으로 가요. 청국장도 덤으로 맛있고 만족...","[1, 1, 0, 1, 1, 0, 1, 1, 1, 1]",NaN
77,메이플라워바베큐,584,4.86,"[(언젠가한량, 색다른 메뉴에 음식도 맛있어서 또 오고 싶어요. 일본 느낌도 물씬 ...","[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ...",NaN
78,호숫가애,416,4.27,"[(kpearl, 저는 누룽지백숙보다는 닭해물탕이 더 맛있었어요ㅎㅎㅎ), (28**...","[1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, ...",NaN


In [211]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
import pandas as pd


In [231]:
features_train = df2[['리뷰 수', '긍정 부정의 비율']]


target_train = df2['별점']

df2_clean = pd.concat([features_train, target_train], axis=1).dropna()


features_predict = df[['리뷰 수', '긍정 부정의 비율']]


X_train, X_test, y_train, y_test = train_test_split(df2_clean[['리뷰 수', '긍정 부정의 비율']], df2_clean['별점'], test_size=0.2, random_state=42)

print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))


model = LinearRegression()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error on Test Set: {mse}')


predicted_ratings = model.predict(features_predict)


df['예측별점'] = predicted_ratings
df['위치(시)'] = '충주'

df

16
16
4
4
Mean Squared Error on Test Set: 0.015789357297345787


,가게 이름,리뷰 수,리뷰 데이터,긍정 부정,긍정 부정의 비율,예측별점,위치(시)
0,돈이 조아,165,"[(홍남매맘맘, 고기는 진짜 끝내줘요 두 번째 갔는데 신랑이 너무 너 무 좋아요 집...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ...",0.490909,4.510530,충주
1,치히로 충주점,1735,"[(낑깡9570, 김치나베돈가스는 보글보글 잘 익은 새콤한 김치찌개 에 맛있는 돈가...","[0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...",0.627273,4.396514,충주
2,삼산회관 충주호암점,211,"[(이성주30, 카이센동이랑 베이컨 크림 우동 먹었는데 둘 다 너무 맛있었어요! ㅠ...","[0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...",0.427273,4.510169,충주
3,카쿠레가 충주점,1017,"[(시봄, 시청 앞 맛집이라고 해서 가봤는데, 짜글이 맛있게 먹었습니다. 고기도 많...","[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, ...",0.781818,4.439100,충주
4,터줏골명가,994,"[(홍남매맘맘, 맛있어요 자극적이지않고 구수하니 !! 양도 옴청 많아 요~~~ 신랑...","[1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, ...",0.881818,4.436276,충주
5,이름난순대의신 본점,486,"[(파란하늘6053, 숙성회가 쫀득하고 맛있어요.), (빈블링, 숙성회 말모말모\n...","[1, 1, 1, 1, 0, 1, 1, 1, 1, 1]",0.900000,4.470422,충주
6,갱고개,199,"[(테테림, 트러플 오일 파스타가 젤 마싯었어요 트러플 향이 은 은하게 나서 좋았어...","[0, 0, 1, 0, 1, 0, 0, 0, 1, 0]",0.300000,4.516603,충주
7,빠리방앗간,383,"[(봄날568, 스테이크 엄청 부드럽고 육즙도 풍부하고 맛있어요. 파스타도 해물도많...","[1, 0, 0, 0, 0, 1, 0, 1, 0, 0]",0.300000,4.503944,충주
8,투파인드피터 충주연수점,734,"[(옌4883, 준경양카츠를 처음 가봤는데, 돈까스가 엄총 바삭바 삭하니 맛있었어요...","[1, 1, 0, 1, 1, 1, 1, 1, 0, 1]",0.800000,4.457767,충주
9,준경양카츠,538,"[(야유유, 고기도 직접 구워주시고 반찬도 잘 나오고 맛있고~ 아 이랑 같이 가니 ...","[0, 1, 0, 1, 1, 1, 0, 0, 0, 0]",0.400000,4.488875,충주
